In [1]:
using SymPy
using LinearAlgebra
@vars J0hat J2hat l1 L1 l2 m1 m2 g b1 b2 t thetadot1 thetadot2 thetaddot1 thetaddot2 tau1 tau2
theta1 = SymFunction("theta1")(t)
theta2 = SymFunction("theta2")(t);

In [2]:
torque1 = diff(diff(theta1,t),t)*(J0hat+J2hat*(sin(theta2))^2)+diff(diff(theta2,t),t)*m2*L1*l2*cos(theta2)-(diff(theta2,t))^2*m2*L1*l2*sin(theta2)+J2hat*diff(theta1,t)*diff(theta2,t)*sin(2*theta2)+b1*diff(theta1,t)
cleantorque1 = torque1.subs([(diff(diff(theta1,t)),thetaddot1),(diff(diff(theta2,t)),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

                                                                 2            
J2̂⋅θ̇₁⋅θ̇₂⋅sin(2⋅θ₂(t)) + L₁⋅l₂⋅m₂⋅θ̈₂⋅cos(θ₂(t)) - L₁⋅l₂⋅m₂⋅θ̇₂ ⋅sin(θ₂(t)) 

               ⎛             2       ⎞
+ b₁⋅θ̇₁ + θ̈₁⋅⎝J0̂ + J2̂⋅sin (θ₂(t))⎠

In [3]:
torque2 = diff(diff(theta1,t),t)*m2*L1*l2*cos(theta2)+J2hat*diff(diff(theta2,t),t)-.5*(diff(theta2,t))^2*J2hat*sin(2*theta2)+b2*(diff(theta2,t))+g*m2*l2*sin(theta2)
cleantorque2 = torque2.subs([(diff(diff(theta1,t)),thetaddot1),(diff(diff(theta2,t)),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

                     2                                                        
J2̂⋅θ̈₂ - 0.5⋅J2̂⋅θ̇₂ ⋅sin(2⋅θ₂(t)) + L₁⋅l₂⋅m₂⋅θ̈₁⋅cos(θ₂(t)) + b₂⋅θ̇₂ + g⋅l₂⋅

             
m₂⋅sin(θ₂(t))

In [4]:
zero1dirty = -tau1 + (diff(diff(theta1,t),t)*(J0hat+J2hat*(sin(theta2))^2)+diff(diff(theta2,t),t)*m2*L1*l2*cos(theta2)-(diff(theta2,t))^2*m2*L1*l2*sin(theta2)+J2hat*diff(theta1,t)*diff(theta2,t)*sin(2*theta2)+b1*diff(theta1,t))
zero1 = zero1dirty.subs([(diff(diff(theta1,t)),thetaddot1),(diff(diff(theta2,t)),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

                                                                 2            
J2̂⋅θ̇₁⋅θ̇₂⋅sin(2⋅θ₂(t)) + L₁⋅l₂⋅m₂⋅θ̈₂⋅cos(θ₂(t)) - L₁⋅l₂⋅m₂⋅θ̇₂ ⋅sin(θ₂(t)) 

                    ⎛             2       ⎞
+ b₁⋅θ̇₁ - τ₁ + θ̈₁⋅⎝J0̂ + J2̂⋅sin (θ₂(t))⎠

## New stuff

Jackson, I added some stuff below here.  Basically, I made up some states to replace each coordinate and used subs to replace them.  The replacement scheme isn't very sophisticated, so I need to be careful on the order of the subs.

I did not check your equations, just wanted to show you the pattern.  I don't know if I'm using the final equations, and I'm unsure of the inputs.

First, I'll setup both equations simultaneously and solve for the 2nd derivative terms:

In [16]:
sol = solve( [torque1, torque2], [diff(theta1,t,t), diff(theta2,t,t)] )

Dict{Any,Any} with 2 entries:
  Derivative(theta2(t), (t, 2)) => (L1*l2*m2*(J2hat*sin(2.0*theta2(t))*Derivati…
  Derivative(theta1(t), (t, 2)) => (-J2hat*(J2hat*sin(2.0*theta2(t))*Derivative…

Now I'll make some states, and replace $[z_1,z_2,z_3,z_4] = [\theta_1,\theta_2,\dot{\theta}_1,\dot{\theta}_2]$

In [17]:
z1,z2,z3,z4,u1,u2 = symbols("z_1,z_2,z_3,z_4,u1, u2")
rule1 = Dict(  diff(theta1,t)=>z3, diff(theta2,t)=>z4 )
rule2 = Dict( theta1=>z1, theta2=>z2, tau1=>u1, tau2=>u2 )

Dict{Sym,Sym} with 4 entries:
  theta2(t) => z_2
  theta1(t) => z_1
  tau2      => u2
  tau1      => u1

Now build $\dot{z} = f(z,u)$  (remember I'm not sure I have the inputs $u$ correct).  I'm going to do this in a verbose way, so you can follow along:

In [26]:
zdot = [diff(theta1,t),diff(theta2,t), diff(theta1,t,t), diff(theta2,t,t)]

4-element Array{Sym,1}:
      Derivative(theta1(t), t)
      Derivative(theta2(t), t)
 Derivative(theta1(t), (t, 2))
 Derivative(theta2(t), (t, 2))

In [27]:
zdot1 = [zd |> subs(sol) for zd in zdot] # this is a loop

4-element Array{Sym,1}:
                                                                                                                                                                                                                                                                                                                                                                                                          Derivative(theta1(t), t)
                                                                                                                                                                                                                                                                                                                                                                                                          Derivative(theta2(t), t)
                           (-J2hat*(J2hat*sin(2.0*theta2(t))*Derivative(theta1(t), t)*Derivative(theta2(t), t) - L1*l2*m2*sin(theta2(t))*D

In [28]:
[ zd |> subs(rule1) for zd in zdot1 ]

4-element Array{Sym,1}:
                                                                                                                                                                                                                                                                                                 z_3
                                                                                                                                                                                                                                                                                                 z_4
                           (-J2hat*(J2hat*z_3*z_4*sin(2.0*theta2(t)) - L1*l2*m2*z_4^2*sin(theta2(t)) + b1*z_3) + 0.5*L1*l2*m2*(-J2hat*z_4^2*sin(2.0*theta2(t)) + 2.0*b2*z_4 + 2.0*g*l2*m2*sin(theta2(t)))*cos(theta2(t)))/(J2hat*(J0hat + J2hat*sin(theta2(t))^2) - L1^2*l2^2*m2^2*cos(theta2(t))^2)
 (L1*l2*m2*(J2hat*z_3*z_4*sin(2.0*theta2(t)) - L1*l2*m2*z_4^2*sin(theta2(t)) + b1*z_3)*cos(theta2

In [30]:
F = [ zd |> subs(rule1) |> subs(rule2) for zd in zdot1 ]

4-element Array{Sym,1}:
                                                                                                                                                                                                                                                 z_3
                                                                                                                                                                                                                                                 z_4
                     (-J2hat*(J2hat*z_3*z_4*sin(2.0*z_2) - L1*l2*m2*z_4^2*sin(z_2) + b1*z_3) + 0.5*L1*l2*m2*(-J2hat*z_4^2*sin(2.0*z_2) + 2.0*b2*z_4 + 2.0*g*l2*m2*sin(z_2))*cos(z_2))/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)
 (L1*l2*m2*(J2hat*z_3*z_4*sin(2.0*z_2) - L1*l2*m2*z_4^2*sin(z_2) + b1*z_3)*cos(z_2) - 0.5*(J0hat + J2hat*sin(z_2)^2)*(-J2hat*z_4^2*sin(2.0*z_2) + 2.0*b2*z_4 + 2.0*g*l2*m2*sin(z_2)))/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)

Now we can linearize

In [38]:
DF = [diff(f,z) for f in F, z in [z1,z2,z3,z4] ]

4×4 Array{Sym,2}:
 0  …                                                                                                                                                                                                                 0
 0                                                                                                                                                                                                                    1
 0                              (-J2hat*(J2hat*z_3*sin(2.0*z_2) - 2*L1*l2*m2*z_4*sin(z_2)) + 0.5*L1*l2*m2*(-2*J2hat*z_4*sin(2.0*z_2) + 2.0*b2)*cos(z_2))/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)
 0     (L1*l2*m2*(J2hat*z_3*sin(2.0*z_2) - 2*L1*l2*m2*z_4*sin(z_2))*cos(z_2) + (-0.5*J0hat - 0.5*J2hat*sin(z_2)^2)*(-2*J2hat*z_4*sin(2.0*z_2) + 2.0*b2))/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)

Now evaluate at equilibrium $z_3 \to 0$ and $z_4 \to 0$.  I don't know if $z_2 \to 0$ or $\pi$, but that should be added here:

In [42]:
A = [df|> subs(z3=>0,z4=>0)|> simplify for df in DF]

4×4 Array{Sym,2}:
 0  …                                                                                                  0
 0                                                                                                     1
 0               1.0*L1*b2*l2*m2*cos(z_2)/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)
 0     -1.0*b2*(J0hat + J2hat*sin(z_2)^2)/(J2hat*(J0hat + J2hat*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)

The last step is to find $B$, which follows a similar pattern.